In [ ]:
Set-up directory/ collect image
- Dataset
  -User 1
    -image 1
    -image 2
    ....
    -image n
  -User 2
    -image 1
    -image 2
    ....
    -image n
  .....
  -User n

In [ ]:
import os
import cv2

def main():
    # dataset path
    base_dir = "D:/AUPP/Junior/Spring 2025/Computer Vision/Final_Project/dataset" # replace your dataset path
    if not os.path.isdir(base_dir):
        print(f"Error: Base folder '{base_dir}' does not exist.")
        return

    # ask for a new, non-existent subfolder
    while True:
        username = input("Enter username: ").strip()
        if not username:
            print("Username cannot be empty.")
            continue
        user_dir = os.path.join(base_dir, username)
        if os.path.exists(user_dir):
            print(f"Subfolder '{username}' already exists. Please choose another.")
        else:
            os.makedirs(user_dir)
            print(f"Created folder: {user_dir}")
            break

    # open webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open webcam.")
        return

    count = 0
    x, y, w, h = 200, 120, 250, 250  # frame region

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        # draw green box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, "Place your face in the box", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        cv2.putText(frame, "Press 's' to save, 'q' to quit", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)

        cv2.imshow("Webcam Capture", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('s'):
            # crop & save
            face = frame[y:y + h, x:x + w]
            fname = f"{username}_{count:03d}.jpg"
            path = os.path.join(user_dir, fname)
            cv2.imwrite(path, face)
            print(f"Saved: {path}")
            count += 1
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Username cannot be empty.
Created folder: D:/AUPP/Junior/Spring 2025/Computer Vision/Final_Project/dataset\a


Data Augmentation

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_images(src_folder, target_folder, num_augmented_images=500, target_size=(100, 100)):
    # Initialize the ImageDataGenerator for augmentation
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    # Iterate through the person folders
    for person_folder in os.listdir(src_folder):
        person_path = os.path.join(src_folder, person_folder)
        if not os.path.isdir(person_path):
            continue
        
        # List all images in the current person's folder
        images = [cv2.imread(os.path.join(person_path, img)) for img in os.listdir(person_path) if img.endswith(('jpg', 'jpeg', 'png'))]
        
        # Create target folder for augmented images
        augmented_person_folder = os.path.join(target_folder, person_folder)
        os.makedirs(augmented_person_folder, exist_ok=True)
        
        # Process each image
        for img in images:
            # Resize image to the target size before augmentation
            img_resized = cv2.resize(img, target_size)
            
            # Expand dimensions to fit the generator
            img_resized = np.expand_dims(img_resized, axis=0)
            
            i = 0
            # Generate augmented images
            for batch in datagen.flow(img_resized, batch_size=1, save_to_dir=augmented_person_folder, save_prefix='aug', save_format='jpg'):
                i += 1
                if i >= (num_augmented_images // len(images)):
                    break

# Usage
augment_images('D:/AUPP/Junior/Spring 2025/Computer Vision/Final_Project/dataset', 'D:/AUPP/Junior/Spring 2025/Computer Vision/Final_Project/augmented_dataset', num_augmented_images=500, target_size=(100, 100))
# augment_images( Path of your dataset, Path of your augmented_dataset, aug number, size)


In [ ]:
Model training

In [ ]:
import os
import itertools
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Build the encoder (shared CNN)
def build_encoder(input_shape=(100, 100, 3)):
    model = models.Sequential(name="embedding")
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(2))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(2))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    return model

# Lambda for absolute difference
@tf.keras.utils.register_keras_serializable()
def compute_abs_difference(tensors):
    return tf.abs(tensors[0] - tensors[1])

# Build Siamese model
def build_siamese(input_shape=(100, 100, 3)):
    encoder = build_encoder(input_shape)
    input1 = layers.Input(shape=input_shape)
    input2 = layers.Input(shape=input_shape)
    feat1 = encoder(input1)
    feat2 = encoder(input2)
    diff = layers.Lambda(compute_abs_difference)([feat1, feat2])
    output = layers.Dense(1, activation='sigmoid')(diff)
    model = models.Model(inputs=[input1, input2], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model, encoder

# Generate training pairs
def generate_pairs(folder, total_pairs_per_person=4000):
    pairs, labels = [], []
    people = [p for p in os.listdir(folder) if os.path.isdir(os.path.join(folder, p))]
    for person in people:
        p_imgs = [os.path.join(folder, person, img) for img in os.listdir(os.path.join(folder, person))]
        positives = list(itertools.combinations(p_imgs, 2))
        positives = random.sample(positives, min(len(positives), total_pairs_per_person // 2))
        negatives = []
        others = [p for p in people if p != person]
        for img1 in p_imgs:
            for op in others:
                op_imgs = [os.path.join(folder, op, img) for img in os.listdir(os.path.join(folder, op))]
                for img2 in op_imgs:
                    negatives.append((img1, img2))
        negatives = random.sample(negatives, min(len(negatives), total_pairs_per_person // 2))
        pairs.extend(positives + negatives)
        labels.extend([1]*len(positives) + [0]*len(negatives))
    return pairs, labels

def preprocess(img_path):
    img = image.load_img(img_path, target_size=(100, 100))
    arr = image.img_to_array(img)
    return preprocess_input(arr)

# Prepare data
dataset_path = "D:/AUPP/Junior/Spring 2025/Computer Vision/augmented_dataset" # replace your augmented dataset path
pairs, labels = generate_pairs(dataset_path, total_pairs_per_person=4000)
X1 = np.array([preprocess(img1) for img1, _ in pairs])
X2 = np.array([preprocess(img2) for _, img2 in pairs])
y = np.array(labels)
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

# Train the model
model, encoder = build_siamese()
history = model.fit([X1_train, X2_train], y_train, batch_size=32, epochs=5, validation_data=([X1_test, X2_test], y_test))

# Save the encoder
encoder.save("face_encoder.h5")


c:\Users\U-ser\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 630s 1s/step - accuracy: 0.5271 - loss: 4.4976 - val_accuracy: 0.6967 - val_loss: 0.6293
Epoch 2/5
 84/500 ━━━━━━━━━━━━━━━━━━━━ 7:47 1s/step - accuracy: 0.5704 - loss: 0.7555

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the encoder
@tf.keras.utils.register_keras_serializable()
def compute_abs_difference(tensors):
    return tf.abs(tensors[0] - tensors[1])

encoder = load_model("C:/Users/U-ser/OneDrive - American University of Phnom Penh/Desktop/Face encode/face_encoder.h5")

In [ ]:
Build encode database and image

In [ ]:
from scipy.spatial.distance import cosine

# Preprocessing & encoding
def load_and_preprocess_image(img_path, target_size=(100, 100)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    return img_array

def get_encoding(image_path, encoder_model):
    img = load_and_preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    encoding = encoder_model.predict(img)
    return encoding[0]

# Build encoding database
def build_encoding_database(dataset_folder, encoder_model):
    encoding_db = {}
    for person in os.listdir(dataset_folder):
        person_path = os.path.join(dataset_folder, person)
        if not os.path.isdir(person_path): continue
        encodings = []
        for img_file in os.listdir(person_path)[:500]:
            if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(person_path, img_file)
                try:
                    enc = get_encoding(img_path, encoder_model)
                    encodings.append(enc)
                except Exception as e:
                    print(f"⚠️ {img_path}: {e}")
        encoding_db[person] = encodings
    return encoding_db

# Compare test image to database
def find_best_match(test_image_path, encoder_model, encoding_db, threshold=0.7):
    test_enc = get_encoding(test_image_path, encoder_model)
    best_score = -1
    best_match = "Unknown"
    for person, enc_list in encoding_db.items():
        for enc in enc_list:
            sim = 1 - cosine(test_enc, enc)
            if sim > best_score:
                best_score = sim
                best_match = person
    print(f"Best Match: {best_match}, Similarity: {best_score:.4f}")
    return best_match if best_score >= threshold else "Unknown"


In [ ]:
import os
import pickle

# Define directory and filename
save_dir = "D:/AUPP/Junior/Spring 2025/Computer Vision/Final_Project" # replace your directory to save encoding database
save_path = os.path.join(save_dir, "face_encoding_database.pkl")

# Make sure the directory exists
os.makedirs(save_dir, exist_ok=True)

# Save
with open(save_path, "wb") as f:
    pickle.dump(encoding_db, f)

print(f"✅ Saved encoding database to: {save_path}")


In [ ]:
import pickle

# ==== RUN TESTING ====
with open("D:/AUPP/Junior/Spring 2025/Computer Vision/Final_Project/face_encoder.h5", "rb") as f: # replace with your encoding database file
    encoding_db = pickle.load(f)
test_img_path = "D:/AUPP/Junior/Spring 2025/Computer Vision/Final_Project/test/Lewis.jpg" # replace with your test image
#'C:/Users/U-ser/Downloads/Telegram Desktop/dataset (2)/dataset/Panha/aug_0_2283.jpg'  Test sample image
result = find_best_match(test_img_path, encoder, encoding_db, threshold=0.7)

if result == "Unknown":
    print("❌ No match found.")
else:
    print(f"✅ Matched with: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Best Match: Lewis, Similarity: 0.7800
✅ Matched with: Lewis


In [48]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from scipy.spatial.distance import cosine
import pickle
import tensorflow as tf
# Load encoder
@tf.keras.utils.register_keras_serializable()
def compute_abs_difference(tensors):
    return tf.abs(tensors[0] - tensors[1])

encoder = load_model("D:/AUPP/Junior/Spring 2025/Computer Vision/Final_Project/face_encoder.h5") # replace with your model

# Load saved encoding database
with open("D:/AUPP/Junior/Spring 2025/Computer Vision/Final_Project/face_encoding_database.pkl", "rb") as f: # replace with your encoding database
    encoding_db = pickle.load(f)

# Preprocess face and get encoding
def get_encoding_from_array(face_array, encoder_model):
    face = cv2.resize(face_array, (100, 100))
    img_array = image.img_to_array(face)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    encoding = encoder_model.predict(img_array)
    return encoding[0]

# Match face to database
def match_face(encoding, encoding_db, threshold=0.8):
    best_score = -1
    best_match = "Unknown"
    for person, encodings in encoding_db.items():
        for known_enc in encodings:
            similarity = 1 - cosine(encoding, known_enc)
            if similarity > best_score:
                best_score = similarity
                best_match = person
    return best_match if best_score >= threshold else "Unknown", best_score

# Load OpenCV Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start Webcam
cap = cv2.VideoCapture(0)

print("🎥 Starting webcam... Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to gray for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_crop = frame[y:y+h, x:x+w]
        encoding = get_encoding_from_array(face_crop, encoder)
        name, score = match_face(encoding, encoding_db, threshold=0.8)

        # Choose color
        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)

        # Draw box and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        label = f"{name} ({score:.2f})" if name != "Unknown" else "Unknown"
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Show frame
    cv2.imshow("Real-Time Face Recognition", frame)

    # Exit key
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


🎥 Starting webcam... Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
